In [ ]:
# GCP cost?
def string_size_in_bytes(s):
    return len(s.encode("utf-8"))


# 23 million bytes
(df.article.apply(string_size_in_bytes).sum() - 1000000) * 0.000016
# ~353.800848 US for all text, less free month allowance


In [ ]:
# coqui price estimates?
from pathlib import Path

from pydub import AudioSegment

# 10 TTS
audio_files = list(Path("./data/tts_output/tacotron2_ddc_ph").rglob("*/*.mp3"))


def get_audio_length(filename):
    audio = AudioSegment.from_file(filename)
    return len(audio)  # length in milliseconds


audio_all = []
for audio_file in audio_files:
    length = get_audio_length(audio_file) / 1000.0  # convert to seconds
    audio_all.append(length)

# hours > https://coqui.ai/pricing > $20/4 hours
sum(audio_all) / 60 / 60
# 2.2028605555555556, yikes


## Coqui model revisions

In [ ]:
from TTS.api import TTS

tts = TTS(model_name="tts_models/en/vctk/vits")


In [ ]:
for speaker in speakers[1:]:
    speaker
    output_file = output_dir / f"vctk_{speaker}_nautilus_editors_note.mp3"
    # tts_coqui_vctk(speaker, nautilus_editors_note, output_file)


In [ ]:
from utils import nautilus_editors_note

nautilus_editors_note


In [ ]:
# code equivlanet for these models?
!tts --text "Behold the humble nautilus. Just about a foot in diameter, it is a slow bottom-dweller with short tentacles that moves through the water with an unsteady wobble. It\'s also 500 million years old and, in its day, was the best and brightest, using its newly evolved depth control to lay waste to acre after acre of scuttling crustacean prey.\nWe became interested in it here at Nautilus because, well, we stole its name. But also because (for a mollusk) it represents a remarkable intersection of science, math, myth, and culture. Since that is exactly the kind of intersection we love to write about, we decided to put together a little "teaser" issue all about it.\nThere\'s the science. The nautilus has a beautiful, logarithmic, and fractal spiral in its shell. Benoit Mandelbrot, discoverer of the fractal, gives us a few words on that topic. One of the world\'s foremost nautilus experts, Peter Ward, tells us about nautilus evolution and biology, and about his life of nautilus research.\nThen, the myth: from Jules Verne\'s fictional submarine, to Oliver Wendell Holmes\' poem, to how and why we turn science into story.\nTwo chapters, one undersea creature. Welcome aboard.\nMichael Segal\n    Editor in Chief"\
--out_path spkr-out.wav --model_name "tts_models/en/vctk/vits" \
--use_cuda True \
--speaker_idx "p227"


## Tortoise Diffusion models

In [ ]:
import sys
from pathlib import Path

import torch
import torchaudio

from tortoise.api import MODELS_DIR, TextToSpeech
from tortoise.utils.audio import load_voices

sys.path.append("../tortoise-tts/tortoise")


voices = """angie                daniel  freeman  jlaw  myself  rainbow       tom           train_dotrice  train_grace     train_mouse
applejack            deniro  geralt   lj    pat     snakes        train_atkins  train_dreams   train_kennard   weaver
cond_latent_example  emma    halle    mol   pat2    tim_reynolds  train_daws    train_empire   train_lescault  william"""


output_dir = Path("./data/tts_output/dev/tortoise")
output_dir.mkdir(parents=True, exist_ok=True)

voices_formatted = [word for line in voices.splitlines() for word in line.split()]
model = voices_formatted[0]

tts = TextToSpeech(models_dir=MODELS_DIR)
voice_samples, conditioning_latents = load_voices([model])


In [ ]:
import spacy
from utils import nautilus_editors_note

nlp = spacy.load("en_core_web_md")

sents = [e.text for e in list(nlp(nautilus_editors_note).sents)]


In [ ]:
for idx, sent in enumerate(sents):
    print(sent)
    gen, dbg_state = tts.tts_with_preset(
        sent,
        k=1,
        voice_samples=voice_samples,
        conditioning_latents=conditioning_latents,
        preset="fast",
        use_deterministic_seed=42,
        return_deterministic_state=True,
        cvvp_amount=0.0,
    )
    torchaudio.save(str(output_dir / f"test_{idx}.wav"), gen.squeeze(0).cpu(), 24000)


In [ ]:
import pandas as pd

from utils import to_snake_case

# re-assign issue/article numbers, whoops
df = (
    pd.read_csv("./data/naut_all.csv")
    .assign(issue_number=lambda x: x.issue_title.factorize()[0] + 1)
    .assign(article_number=lambda x: x.groupby("issue_number").cumcount() + 1)
)


## Conversion estimates

In [ ]:
dff = pd.read_csv("./data/tts_logs.csv")

median_token_conversion = (dff.time_elapsed / dff.n_tokens).median()
median_token_conversion  # median time-per-token 0.007610222857264401 per token


In [ ]:
print((dff.assign(token_estimate=lambda x: x.n_tokens * median_token_conversion)))

#    issue_number  article_number  n_tokens  time_elapsed  token_estimate
# 0             1               1       199      7.813745        1.514434
# 1             1               2      3129     29.584105       23.812387
# 2             1               3      4239     31.648683       32.259735
# 3             1               4      2033     14.212531       15.471583
# 4             1               5      1928     14.536405       14.672510
# 5             1               6      2662     18.884738       20.258413
# 6             2               1       355      5.581702        2.701629
# 7             2               2      1478     11.352247       11.247909
# 8             2               3      3912     29.144785       29.771192
# 9             2               4       654      6.788820        4.977086


(
    df.article.apply(lambda x: len(x.split(" "))) * median_token_conversion
).sum() / 60 / 60


## S3 streaming?

In [ ]:
import pandas as pd

# ingest all articles
df = (
    pd.read_csv("./data/naut_all.csv")
    .assign(issue_number=lambda x: x.issue_title.factorize()[0] + 1)
    .assign(article_number=lambda x: x.groupby("issue_number").cumcount() + 1)
)

In [ ]:
from urllib.parse import quote

# Extract the S3 bucket and key from the S3 path
s3_path = f"s3://{BUCKET_NAME}/{dff.iloc[0].audio_url}"
bucket = s3_path.split("//")[1].split("/")[0]
key = "/".join(s3_path.split("//")[1].split("/")[1:])

# Generate a valid URL for redirection
redirect_url = f"https://{bucket}.s3.amazonaws.com/{quote(key)}"

In [ ]:
bucket

In [ ]:
BUCKET = 'blog-tts-pod'


In [ ]:
import boto3
import pandas as pd
from io import StringIO

# Create an S3 client
s3 = boto3.client('s3')

# These define the bucket and object to read
BUCKET_NAME = "blog-tts-pod"
NAUT_ALL_OBJECT_KEY = "data/naut_all.csv"

# Use the S3 client to read the file
object_data = s3.get_object(Bucket=BUCKET_NAME, Key=NAUT_ALL_OBJECT_KEY)
file_data = object_data['Body'].read()

# Convert the file data to a pandas DataFrame
data = pd.read_csv(StringIO(file_data.decode('utf-8')))
